In [ ]:
# Create an IAM role called [SageMakerRole] with AmazonSageMakerFullAccess and AmazonEC2ContainerRegistryFullAccess manually
# Add the IAM role of the notebook with AmazonEC2ContainerRegistryFullAccess mannually

# Create MXNet Container

In [ ]:
# Create the binary
!git clone https://github.com/aws/sagemaker-mxnet-container.git
!cd sagemaker-mxnet-container
!git checkout v3.1.2
!python setup.py sdist

In [ ]:
# MXNet 1.4.1, Python 3, CPU
!cp dist/sagemaker_mxnet_container-3.1.2.tar.gz docker/1.4.1/py3/sagemaker_mxnet_container.tar.gz
!cd docker/1.4.1/py3/
!docker build -t preprod-mxnet:1.4.1-cpu-py3 -f Dockerfile.cpu .

In [ ]:
!cd ../../../
!pip install -e .[test]

In [ ]:
# May fail
!tox test/unit

In [ ]:
# May fail
!tox -- test/integration/local --docker-base-name preprod-mxnet \
                              --tag 1.4.1-cpu-py3 \
                              --py-version 3 \
                              --framework-version 1.4.1 \
                              --processor cpu

In [4]:
# upload container to ECS
# create-repository in ECR mannually
!$(aws ecr get-login --region ap-northeast-1 --no-include-email)
!docker tag preprod-mxnet:1.4.1-cpu-py3 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet:1.4.1-cpu-py3
!docker push 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet:1.4.1-cpu-py3

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: login attempt to https://579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/v2/ failed with status: 400 Bad Request
The push refers to repository [579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet]

c6cf436c: Preparing 
a47b956c: Preparing 
0ac446f2: Preparing 
09efe46f: Preparing 
368178b1: Preparing 
7ba6b896: Preparing 
fb893158: Preparing 
42719515: Preparing 
103e78c9: Preparing 
e637fbff: Preparing 
fb893158: Pushed   668.8MB/654.1MB1A1.4.1-cpu-py3: digest: sha256:9e388fdb9a291d4fc51adc682761477c677cc676ed22a28de240626e1f7365cb size: 2622


In [5]:
# May fail
!tox -- test/integration/sagemaker --aws-id 579019700964 \
                                  --docker-base-name preprod-mxnet \
                                  --instance-type ml.m4.xlarge \
                                  --tag 1.4.1-cpu-py3

/bin/sh: tox: command not found


# Create TextClassification Train Docker

In [ ]:
!cd gluon-nlp/model_zoo/text_classification/

!aws s3 cp --recursive s3://app-mia/mytask/ .

# create train

# create hyperparameters.json

# create inputdataconfig.json

# create Dockerfile.train

chmod +x build_and_push.sh
./build_and_push.sh text-classification-train train

!docker run text_classification_train train

# Test TextClassification Train Docker

In [20]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'text-classification-train'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/text-classification-train:latest


In [22]:
import json
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role, session

role = get_execution_role()

hyperparameters = json.load(open('/home/ec2-user/SageMaker/gluon-nlp/model_zoo/text_classification/hyperparameters.json', 'r'))

bucket = 'app-mia'
s3_train_data = 's3://{}/{}'.format(bucket, 'mytask/train.tsv')
s3_validation_data = 's3://{}/{}'.format(bucket, 'mytask/dev.tsv')
train_data = session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

instance_type = 'ml.m4.xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_channels)  # data_channels

2019-08-30 00:08:55 Starting - Starting the training job...
2019-08-30 00:08:57 Starting - Launching requested ML instances......
2019-08-30 00:10:00 Starting - Preparing the instances for training...
2019-08-30 00:10:54 Downloading - Downloading input data
2019-08-30 00:10:54 Training - Downloading the training image......
2019-08-30 00:11:55 Training - Training image download completed. Training in progress............
Training complete.

2019-08-30 00:13:48 Uploading - Uploading generated training model
2019-08-30 00:13:48 Completed - Training job completed
Billable seconds: 181


# Create MXNet Serving Container

In [ ]:
!git clone https://github.com/aws/sagemaker-mxnet-serving-container.git
!cd sagemaker-mxnet-serving-container
!git checkout v1.1.3
!python setup.py sdist

In [ ]:
!cp dist/sagemaker_mxnet_serving_container-1.1.3.tar.gz docker/1.4.1/py3/sagemaker_mxnet_serving_container.tar.gz
!cd docker/1.4.1/py3/
!docker build -t preprod-mxnet-serving:1.4.1-cpu-py3 -f Dockerfile.cpu .

In [ ]:
!cd ../../../
!pip install -e .[test]

In [ ]:
!tox test/unit

In [ ]:
# May fail
!tox test/integration/local -- --docker-base-name preprod-mxnet-serving \
                              --tag 1.4.1-cpu-py3 \
                              --py-version 3 \
                              --framework-version 1.4.1 \
                              --processor cpu

In [7]:
# upload container to ECS
# create-repository in ECR mannually
!$(aws ecr get-login --region ap-northeast-1 --no-include-email)
!docker tag preprod-mxnet-serving:1.4.1-cpu-py3 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet-serving:1.4.1-cpu-py3
!docker push 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet-serving:1.4.1-cpu-py3

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet-serving]

ffc7df91: Preparing 
e6a1488e: Preparing 
3622cc17: Preparing 
09f54a8b: Preparing 
8dcd09a2: Preparing 
8eb1a49d: Preparing 
e4aff054: Preparing 
5a9777d0: Preparing 
653357e1: Preparing 
642a33d9: Preparing 
42719515: Preparing 
103e78c9: Preparing 
e637fbff: Preparing 
642a33d9: Pushed   794.3MB/780.7MB12A1.4.1-cpu-py3: digest: sha256:a25aa8a2d83ac28722f4fb178619f43b17fb6cc8bdc769ddc7ed8ca82a036da8 size: 3245


In [ ]:
# May fail
!tox test/integration/sagemaker -- --aws-id 579019700964 \
                                  --docker-base-name preprod-mxnet-serving \
                                  --instance-type ml.m4.xlarge \
                                  --tag 1.4.1-cpu-py3

# Create TextClassification Serving Docker

In [ ]:
!cd gluon-nlp/model_zoo/text_classification/

# create serve

# create Dockerfile.serve

chmod +x build_and_push.sh
./build_and_push.sh text-classification-serve serve

!docker run text_classification_serve serve